### Instructions for Running the Notebook

This notebook is designed to be executed on the JHTDB Sciserver platform. To use it:

1. Create an account on JHTDB Sciserver.
2. Place this script in the folder:
   ```
   /home/idies/workspace/Storage/username/persistent
   ```
   Replace `username` with your actual username.
3. Add the file `parameters.txt` to the same folder. This file must include a key obtained by contacting the dataset owners via email, as specified on their website.
4. The results will be placed in the folder:
   ```
   /home/idies/workspace/Temporary/username/results
   ```
   Replace `username` with your actual username in the notebook.
### Purpose
This script downloads 1000 files, each containing a flow field of shape `[2048, 100, 1536, 3]` for a single time step. Files are named `channel_full_ts_{i}`, where `i` represents the time step.

In [15]:
"""Script to download the required JHTDB dataset"""
import numpy as np
import pyJHTDB
from pyJHTDB import libJHTDB
import ctypes
import os

os.chdir('/home/idies/workspace/Storage/username/persistent')

In [16]:
import json
with open("parameters.txt","r") as pf:
    params=json.load(pf)

auth_token=params["token"]
tstart=int(params.get("ts"))
tend=int(params.get("te"))
tstep=int(params.get("stridet"))
xstart=int(params.get("xs"))
ystart=int(params.get("ys"))
zstart=int(params.get("zs"))
xend=int(params.get("xe"))
yend=int(params.get("ye"))
zend=int(params.get("ze"))
xstep=int(params.get("stridex",1))
ystep=int(params.get("stridey",1))
zstep=int(params.get("stridez",1))
Filter_Width=int(params.get("filter_width",1))
time_step=int(params.get("stridet",0))
fields=params.get("function","u")
data_set=params.get("dataset","isotropic1024coarse")
output_filename=params.get("output_filename",data_set)

In [ ]:
os.chdir('/home/idies/workspace/Temporary/username/results')

In [18]:
lJHTDB = libJHTDB()
lJHTDB.initialize()
lJHTDB.lib.turblibSetExitOnError(ctypes.c_int(0));
lJHTDB.add_token(auth_token)

## "filename" parameter is the file names of output files, if filename='N/A', no files will be written. 
##             For example, if filename='results', the function will write "results.h5" and "results.xmf".
## The function only returns the data at the last time step within [t_start:t_step:t_end]
## The function only returns the data in the last field. For example, result=p if field=[up].
for t in range(1000):
    full_filename = output_filename + f"_{t:04}"
    result = lJHTDB.getbigCutout(
            data_set=data_set, fields=fields, t_start=tstart+t, t_end=tend+t, t_step=tstep,
            start=np.array([xstart, ystart, zstart], dtype = np.int),
            end=np.array([xend, yend, zend], dtype = np.int),
            step=np.array([xstep, ystep, zstep], dtype = np.int),
            filter_width=Filter_Width,filename=full_filename)

lJHTDB.finalize()
print(result.shape)

KeyboardInterrupt: 